# Conv1D

In [2]:
############ IMPORTS ####################
import sys
sys.path.append('./../')
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv1D, Dense, Flatten, MaxPooling1D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical as keras_to_categorical

############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(bands, num_class):
    clf = Sequential()
    clf.add(Conv1D(20, (24), activation='relu', input_shape=(patchsize1*patchsize2,bands)))
    clf.add(MaxPooling1D(pool_size=5))
    clf.add(Flatten())
    clf.add(Dense(100))
    clf.add(BatchNormalization())
    clf.add(Activation('relu'))
    clf.add(Dense(num_class, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return clf
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))
    
    for iter in range(3):
        clf = get_model_compiled(NC, Classes)
        clf.summary()
        valdata = (TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3]).cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1]*TrainPatch1.shape[2],TrainPatch1.shape[3]).cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv1D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv1D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3]).cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv1D_Report_' + datasetName +'.txt')

    

----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 98, 20)            30260     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 19, 20)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 380)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               38100     
_________________________________________________________________
batch_normalization_1 (Batch (N

Epoch 48/200
4/4 [==============================] - 0s 108ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3218 - val_accuracy: 0.9080
Epoch 49/200
1/4 [======>.......................] - ETA: 0s - loss: 0.0052 - accuracy: 1.0000

KeyboardInterrupt: 

# Conv2D

In [8]:
############ IMPORTS ####################
import sys
sys.path.append('./../')
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import to_categorical as keras_to_categorical

############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
datasetNames = ["Trento"]
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(shapeinput, num_class, w_decay=0):
    clf = Sequential()
    clf.add(Conv2D(50, kernel_size=(5, 5), input_shape=shapeinput))
    clf.add(Activation('relu'))
    clf.add(Conv2D(100, (5, 5)))
    clf.add(Activation('relu'))
    clf.add(MaxPooling2D(pool_size=(2, 2)))
    clf.add(Flatten())
    clf.add(Dense(100, kernel_regularizer=regularizers.l2(w_decay)))
    clf.add(Activation('relu'))
    clf.add(Dense(num_class, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return clf

        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))
    
    for iter in range(3):
        inputshape = TrainPatch1.shape[1:]
        clf = get_model_compiled(inputshape, Classes, w_decay = 0.02)
        clf.summary()
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv2D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv2D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv2D_Report_' + datasetName +'.txt')

    

----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 7, 7, 50)          78800     
_________________________________________________________________
activation_7 (Activation)    (None, 7, 7, 50)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 100)         125100    
_________________________________________________________________
activation_8 (Activation)    (None, 3, 3, 100)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (N

KeyboardInterrupt: 

# Conv3D

In [ ]:
############ IMPORTS ####################
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv3D, Dense, Flatten, MaxPooling3D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import to_categorical as keras_to_categorical


############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
datasetNames = ["Trento"]
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(shapeinput, num_class, w_decay=0, lr=1e-3):
            clf = Sequential()
            clf.add(Conv3D(32, kernel_size=(5, 5, 24), input_shape=shapeinput))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(Conv3D(64, (5, 5, 16)))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(MaxPooling3D(pool_size=(2, 2, 1)))
            clf.add(Flatten())
            clf.add(Dense(300, kernel_regularizer=regularizers.l2(w_decay)))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(Dense(num_class, activation='softmax'))
            clf.compile(loss=categorical_crossentropy, optimizer=Adam(lr=lr), metrics=['accuracy'])
            return clf
        
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))

    for iter in range(3):
        inputshape = TrainPatch1.shape[1:]
        clf = get_model_compiled(inputshape, Classes, w_decay = 0.02)
        clf.summary()
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv3D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv3D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv3D_Report_' + datasetName +'.txt')

    